In [1]:
"Timetable parsing"

from tramnetwork import *

lookup_day = "20250128"

network = TramNetwork()
network.load_day(lookup_day)

loaded routes.txt
loaded transfers.txt
loaded stops.txt
loaded trips.txt
loaded stop_times.txt
loaded calendar.txt
loaded calendar_dates.txt
Found 203 Tram Stops (20250128).          


In [2]:
"test timetable"

search_query = "Letzi"

time = gt_parse_time("12:00:00", lookup_day)
stop = network.search_stops(search_query)[0]
departures = stop.get_departures_after(time)
first_departure = departures[0]

print(f"Next Departure from {stop.name} is at {first_departure.departure_time} with {first_departure.tram_name}:")
print(first_departure)

Next Departure from Zürich, Letzistrasse is at 2025-01-28 12:00:18 with T9:
[2025-01-28 12:00:06] Zürich, Letzistrasse           (T9) [2025-01-28 12:00:18]
[2025-01-28 12:00:54] Zürich, Kinkelstrasse          (T9) [2025-01-28 12:01:06]
[2025-01-28 12:02:00] Zürich, Seilbahn Rigiblick     (T9) [2025-01-28 12:02:18]
[2025-01-28 12:03:12] Zürich, Winkelriedstrasse      (T9) [2025-01-28 12:03:24]
[2025-01-28 12:04:18] Zürich, Haldenbach             (T9) [2025-01-28 12:04:30]
[2025-01-28 12:05:42] Zürich, ETH/Universitätsspital (T9) [2025-01-28 12:06:00]
[2025-01-28 12:07:18] Zürich, Kantonsschule          (T9) [2025-01-28 12:07:30]
[2025-01-28 12:08:48] Zürich, Kunsthaus              (T9) [2025-01-28 12:09:06]
[2025-01-28 12:11:00] Zürich, Bellevue               (T9) [2025-01-28 12:11:30]
[2025-01-28 12:13:06] Zürich, Bürkliplatz            (T9) [2025-01-28 12:13:24]
[2025-01-28 12:14:36] Zürich, Kantonalbank           (T9) [2025-01-28 12:14:54]
[2025-01-28 12:15:48] Zürich, Paradeplatz   

In [19]:
"pathfinding using djikstra"

import bisect

MAX_TRANSITION_SECONDS = 30 * 60
MIN_CHANGE_BUFFER_SECONDS = 30

def calc_dijkstra_path_to(start_time: datetime, start: TramStop, destination_criterium,
                          start_tram: TramName=None, stop_weights: dict[TramStop, float]={}) -> TramPath:
    visited_stops = set()
    start_connection = TramPath([start], [start_time], [None], [None])
    visit_stack = [(0, start_time, start_connection, start)]

    while len(visit_stack) > 0:
        _, curr_time, prev_stops, stop = visit_stack.pop(0)
        if stop in visited_stops:
            continue

        visited_stops.add(stop)

        connections = stop.get_departures_after(curr_time)
        for connection in connections:
            wait_seconds = (connection.arrival_time - curr_time).total_seconds()
            
            if len(prev_stops) >= 2:
                last_tram_name = prev_stops.transportation_names[-2]
            else:
                last_tram_name = start_tram

            if last_tram_name is not None and connection.tram_name != last_tram_name:
                # it's a change of tram!
                if wait_seconds < MIN_CHANGE_BUFFER_SECONDS:
                    continue

            if wait_seconds >= MAX_TRANSITION_SECONDS:
                break

            new_stop = connection.stops[1]
            new_time = connection.arrival_times[1]
            if new_stop in visited_stops:
                continue

            prev_stops_copy = prev_stops.slice(0)
            prev_stops_copy.stop_departure_times[-1] = connection.departure_time
            prev_stops_copy.transportation_names[-1] = connection.tram_name

            prev_stops_copy.add_stop(new_stop, new_time, None, None)

            if destination_criterium(new_stop):
                return prev_stops_copy
            
            weight = stop_weights[new_stop] if new_stop in stop_weights else 0
            bisect.insort(visit_stack, (weight, new_time, prev_stops_copy, new_stop))
            
    print(f"{visited_stops=}")
    raise Exception("Couldn't find connection in time")

def calc_dijkstra_path_between(start_time: datetime, start: TramStop, destination: TramStop,
                               start_tram: TramName=None, stop_weights: dict[TramStop, float]={}) -> TramPath:
    destination_criterium = lambda stop: stop == destination
    return calc_dijkstra_path_to(start_time, start, destination_criterium, start_tram=start_tram, stop_weights=stop_weights)

stop1 = network.search_stops("Albis")[0]
stop2 = network.search_stops("Flug")[0]
start_time = gt_parse_time("12:00:00", lookup_day)

connection = calc_dijkstra_path_between(start_time, stop1, stop2)
connection.print_summary()

Start: Zürich, Albisgütli
Destination: Zürich Flughafen, Bahnhof
Total Time: 0:52:18
Start Time: 2025-01-28 12:03:00
Total Stops: 35


In [20]:
"nearest neighbour search"

def calc_nearest_neighbor_search(start_stop: TramStop, start_time: datetime, stop_weights: dict[TramStop, float]={}) -> TramPath:
    unvisited_stops = set(network.stops)
    unvisited_stops.remove(start_stop)
    destination_criterium = lambda stop: stop in unvisited_stops
    visit_count = 0

    curr_stop = start_stop
    curr_time = start_time
    path = TramPath()
    curr_tram = None
    while len(unvisited_stops) > 0:
        new_path = calc_dijkstra_path_to(curr_time, curr_stop, destination_criterium, curr_tram, stop_weights=stop_weights)
        curr_time = new_path.arrival_times[-1]

        visit_count += 1
        path.add_path(new_path)
        curr_stop = path.stops[-1]
        curr_tram = path.transportation_names[-2]
        unvisited_stops.remove(curr_stop)

    return path

letzi = network.search_stops("Letzi")[0]
start_time = gt_parse_time("10:00:00", lookup_day)

nearest_neighbour_path = calc_nearest_neighbor_search(letzi, start_time)
nearest_neighbour_path.print_summary()

Start: Zürich, Letzistrasse
Destination: Zürich, Frankental
Total Time: 11:00:24
Start Time: 2025-01-28 10:00:18
Total Stops: 364


In [21]:
"test loading old path"

genetic_path = TramPath.load("genetic.2025-01-28--9-10-36.txt", network)
genetic_path.print_summary()

Start: Zürich, Letzistrasse
Destination: Schlieren, Geissweid
Total Time: 9:10:36
Start Time: 2025-01-28 09:45:24
Total Stops: 314


In [22]:
"calc stop-heuristics for later optimization"

heuristic_map: dict[TramStop, dict[str, float]] = {}
neighbour_map: dict[TramStop, set[TramStop]] = {}

min_connections = min(len(s.connections) for s in network.stops)
max_connections = max(len(s.connections) for s in network.stops)

for stop in network.stops:
    heuristics = {}
    neighbour_map[stop] = set(c.stops[1] for c in stop.connections) - set([stop])
    heuristics["popularity"] = len(neighbour_map[stop])
    heuristics["connectivity"] = (len(stop.connections) - min_connections) / (max_connections - min_connections)
    heuristics["spider-leg"] = 1 if heuristics["popularity"] == 1 else 0
    heuristics["distance_sum"] = sum(
        1 / (haversine_distance(stop.coords, other.coords) * 10) ** 2
        for other in network.stops if stop != other 
    )
    heuristic_map[stop] = heuristics

for i in range(len(network.stops)):
    for stop in network.stops:
        neighbours = neighbour_map[stop]
        if len(neighbours) != 2:
            continue
        if any(heuristic_map[n]["spider-leg"] for n in neighbours):
            heuristic_map[stop]["spider-leg"] = 1

heuristic_names = list(list(heuristic_map.values())[0].keys())

In [56]:
"optimize heuristic weights to find optimal nearest neighbour search path"

import random, numpy
from functools import cached_property

start_stop = network.search_stops("Letzi")[0]
start_time = gt_parse_time("10:00:00", lookup_day)

MIN_TIME_DELTA = timedelta(hours=9)

class StopWeighting:

    def __init__(self, weights=None):
        if weights is None:
            self.weights = tuple(0 for _ in range(len(heuristic_names)))
        else:
            self.weights = tuple(weights)
            assert len(self.weights) == len(heuristic_names)

    def mutate(self, mutation_chance: float, mutation_rate: float) -> "StopWeighting":
        new_weights = self.weights[:]
        for i in range(len(self.weights)):
            if random.random() < mutation_chance:
                new_weights[i] = self.weights[i] + numpy.random.normal(0, 1) * mutation_rate
        return StopWeighting(new_weights)
    
    def __repr__(self):
        return f"{self.__class__.__name__}({self.weights!r})"
    
    def __str__(self):
        return repr(self)

    def copy(self):
        return StopWeighting(self.weights[:])
    
    @classmethod
    def zero(cls):
        return StopWeighting()
    
    @classmethod
    def random(cls):
        return StopWeighting([
            numpy.random.normal(0, 1)
            for _ in range(len(heuristic_names))
        ])

    def make_map(self) -> dict[TramStop, float]:
        return {
            stop: sum(
                self.weights[i] * heuristic_map[stop][h]
                for i, h in enumerate(heuristic_names)
            )
            for stop in network.stops
        }
    
    @cached_property
    def weighted_path(self) -> TramPath:
        try:
            return calc_nearest_neighbor_search(
                start_stop, start_time, self.make_map()
            )
        except KeyboardInterrupt:
            raise
        except:
            return None
    
    def calc_score(self) -> float:
        weighted_path = self.weighted_path
        if weighted_path is None:
            delta = weighted_path.time_delta() - MIN_TIME_DELTA
            return 1 / delta.total_seconds()
        else:
            return 0
    
    def __hash__(self):
        return hash(self.weights)

population = [
    StopWeighting.random()
    for i in range(100)
]

best_genetic_path = None
best_genetic_delta = None

for i in range(1):
    print(f"Gen{i}: {best_genetic_delta}, {population[0]}")

    # evaluation
    evaluations = {
        weighting: weighting.calc_score()
        for weighting in population
    }

    # selection
    population.sort(key=lambda w: evaluations[w], reverse=True)
    parents = population[:10]
    best = population[0]

    # update global best
    if best_genetic_delta is None or best.weighted_path.time_delta() < best_genetic_delta:
        best_genetic_delta = best.weighted_path.time_delta()
        best_genetic_path = best.weighted_path

    # reproduction and mutation
    for i in range(10, len(population)):
        random_parent = random.choice(parents)
        population[i] = random_parent.mutate(0.3, 0.3)

Gen0: None, StopWeighting((-1.8417432351385952, 0.5698676442476639, -0.7457304917853452, -0.35602946522534634))


TypeError: '<' not supported between instances of 'datetime.timedelta' and 'NoneType'

In [53]:
best_weighting

StopWeighting([-1.0529468945117268, -0.2640859471180964, 0.38084221291094267, -1.102662522791521])